http://qiita.com/Tama_maru/items/26a0509d2e30caa4a975

http://qiita.com/daisukeokaoss/items/738566e9f24d114651ab

http://qiita.com/wwacky/items/98d8be2844fa1b778323

## MacBookへのOpenCVのインストール法 (これが最初大変だった)

2016/6/17(日)　15日(木)夜からの苦戦の末、opencv3のインストールに成功。brewをつかう方法など色々参照したが分かりにくく、結局成功した手順は、pyenvで入れたAnacondaにインストールする方法。

・Anaconda最新版(anaconda3-4.3.1)をpyenv介してMacBookにインストール。
http://qiita.com/keitaMatsuo/items/900e549769484363fba2

・OpenCV3が3.6以降に対応していないため、pythonは$conda install python=3.5.2で最新版(3.6-)よりダウングレード。
http://nnamm.com/blog/1336

・pyenvとAnacondaのactivateが衝突するので、下記ページ2.の方法で回避策。anacondaのversionは3-4.3.1にして設定。
http://qiita.com/y__sama/items/f732bb7bec2bff355b69

・そのうえで、仮想環境でopencv3インストール。http://nnamm.com/blog/1336
$conda create -n py35opencv python=3.5 anaconda

$source ~/.bashrc py35opencv (一つ上のメモ参照)

$conda install --channel https://conda.anaconda.org/menpo opencv3

$conda listでanacondaのパッケージとライブラリの一覧表示

・デスクトップのAnaconda Navigatorアイコンからではなく、ターミナルからjupyter notebookと打って起動すると、cv3の入ったanacondaのJupyter Notebookが動く。試しに、$import cv2とすればopencv3が実行されること確認できる(opnecv3なのににimportはcv2であること注意)。

・ちなみに「Macにpythonとanacondaの環境を構築してみる（pyenv,condaにて）」http://qiita.com/keitaMatsuo/items/900e549769484363fba2

In [ ]:
import cv2

In [ ]:
import cv2#上記のOpenCVインストール実行後であれば走ること確認できる。
img = cv2.imread('Lenna.png',0)
cv2.imshow('img',img)
cv2.waitKey(0)
#上記実行するとLennaの写真が出る。Kernel から　Restart & Clear Outputとすると写真消せる。 

In [ ]:
import cv2#上記のOpenCVインストール実行後であれば走ること確認できる。
img = cv2.imread('yoshinobu.jpg',1)
cv2.imshow('img2',img)
cv2.waitKey(0)
#上記実行すると徳川慶喜の写真が出る。Kernel から　Restart & Clear Outputとすると写真消せる。 

## 人物写真から顔画像の部分だけを四角で囲む

顔探索用のカスケード型分類器のファイル"haarcascade_frontalface_alt.xml"は以下からDL可能。
https://ja.osdn.net/projects/sfnet_magicvisionport/downloads/mvp/cascades/haarcascade_frontalface_alt.xml/

<以下のシェルのコードで参考にしたサイト>
https://www.blog.umentu.work/python-opencv3%E3%81%A7%E7%94%BB%E5%83%8F%E3%81%AE%E9%A1%94%E5%88%A4%E5%AE%9A%E3%83%8D%E3%82%BF%E3%81%82%E3%82%8A/

In [ ]:
#徳川慶喜の写真
#sourceディレクトリにface.pyを作成せよというのが元ネタだったが、書き換えた。

# -*- coding: UTF-8 -*- 
 
import cv2
import math
import numpy as np
import os
 
#if __name__ == '__main__':
 
# 顔判定で使うxmlファイルを指定する。顔探索用カスケード型分類器haarcascade_frontalface_alt.xml同じフォルダに置いた。
#cascade = cv2.CascadeClassifier("/Users/goji/scraping_data/haarcascade_frontalface_alt.xml")
cascade = cv2.CascadeClassifier("/haarcascade_frontalface_alt.xml")
 
# 画像の読み込み。有名なLennaさんの写真使った。同じフォルダに置いた。
#img_src = cv2.imread("Lenna.png", 1)
img_src = cv2.imread("yoshinobu.jpg", 1)
 
# 結果を保存するための変数を用意しておく。
img_result = img_src
 
# グレースケールに変換
img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)
 
#顔判定
""" 
minSize で顔判定する際の最小の四角の大きさを指定できる。
(小さい値を指定し過ぎると顔っぽい小さなシミのような部分も判定されてしまう。)
"""
faces  =  cascade.detectMultiScale(img_gray, scaleFactor=1.1, minNeighbors=1, minSize=(100, 100))
 
# 顔があった場合
if len(faces) > 0:
 
#顔認識の枠の色
    color = (255, 0, 0)
 
# 複数の顔があった場合、１つずつ四角で囲っていく
for face in faces:
 
    # faceには(四角の左上のx座標, 四角の左上のy座標, 四角の横の長さ, 四角の縦の長さ) が格納されている。
 
    # 囲う四角の左上の座標
    coordinates = tuple(face[0:2])
    # (囲う四角の横の長さ, 囲う四角の縦の長さ)
    length = tuple(face[0:2] + face[2:4])
 
    # 四角で囲う処理
    cv2.rectangle(img_result, coordinates, length, color, thickness=3)
 
 
    # 表示
    cv2.imshow("Show FACES Image", img_result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 人物写真からの顔画像の枠囲み&当該顔画像のみの保存("face_of_元写真名")

http://rasp.hateblo.jp/entry/2016/01/22/201534
PythonとOpenCVで画像処理①【読み込み、表示、保存】

In [ ]:
# -*- coding: utf-8 -*-

import cv2

#顔のカスケード分類器のファイルを同じディレクトリに入れてパス指定
cascade_path = "haarcascade_frontalface_alt.xml"

#カスケード分類器の特徴量を取得する
cascade = cv2.CascadeClassifier(cascade_path)

#対象の人物の写真を入れる。デフォルトは有名なLennaさんと、徳川慶喜。
photo_name = "Lenna.png"
#photo_name = "yoshinobu.jpg"

img = cv2.imread(photo_name)
#img = cv2.imread('Lenna.png')

# 顔を検知
faces = cascade.detectMultiScale(img, minSize=(100,100))#顔miniサイズを追加

#このコードでは、一つの写真には一人だけ写っている前提
for (x,y,w,h) in faces:
    # 検知した顔を矩い青の四角形で囲む
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    #顔の部分だけのカット写真を作る。
    cut_img = img[y:y+h,x:x+w]

# 画像表示
cv2.imshow('img',img)

# 以下、「PythonとOpenCVで画像処理①【読み込み、表示、保存】
#http://rasp.hateblo.jp/entry/2016/01/22/201534」を参考に記述。

#キーが押されるまで画像を表示したままにする。
#戻り値: 押されたキーコード
keycode = cv2.waitKey(0)

# "S"を押すと、face_of_写真名というファイル名で顔の部分の写真を保存する。
if keycode == ord('s'):
    #画像を保存
     #第一引数: 保存するファイル名
     #第一引数: 保存したい画像
    cv2.imwrite("face_of_" + photo_name,cut_img)

# 作成したウィンドウをすべて破棄
cv2.destroyAllWindows()